[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/5_train_preference_comparisons.ipynb)
# Learning a Reward Function using Preference Comparisons

The preference comparisons algorithm learns a reward function by comparing trajectory segments to each other.

To set up the preference comparisons algorithm, we first need to set up a lot of its internals beforehand:

In [1]:
import random
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
import gym
from stable_baselines3 import PPO
import numpy as np

rng = np.random.default_rng(0)

venv = make_vec_env("Pendulum-v1", rng=rng)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(
    warning_threshold=0,
    rng=rng,
)
querent = preference_comparisons.PreferenceQuerent()
gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
preference_model = preference_comparisons.PreferenceModel(reward_net)
reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=preference_comparisons.CrossEntropyRewardLoss(),
    epochs=3,
    rng=rng,
)

agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=0,
    n_steps=2048 // venv.num_envs,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=agent,
    reward_fn=reward_net,
    venv=venv,
    exploration_frac=0.0,
    rng=rng,
)

pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=5,
    fragmenter=fragmenter,
    preference_querent=querent,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    fragment_length=100,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    initial_epoch_multiplier=1,
)

Then we can start training the reward model. Note that we need to specify the total timesteps that the agent should be trained and how many fragment comparisons should be made.

In [2]:
pref_comparisons.train(
    total_timesteps=5_000,  # For good performance this should be 1_000_000
    total_comparisons=200,  # For good performance this should be 5_000
)

Query schedule: [20, 51, 41, 34, 29, 25]
Requested 4000 transitions but only 0 in buffer. Sampling 4000 additional transitions.
Creating fragment pairs
Gathering preferences
Dataset now contains 20 comparisons


Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 1000 timesteps
----------------------------------------------------
| raw/                                 |           |
|    agent/rollout/ep_len_mean         | 200       |
|    agent/rollout/ep_rew_mean         | -1.31e+03 |
|    agent/rollout/ep_rew_wrapped_mean | -31.8     |
|    agent/time/fps                    | 2404      |
|    agent/time/iterations             | 1         |
|    agent/time/time_elapsed           | 0         |
|    agent/time/total_timesteps        | 2048      |
----------------------------------------------------
------------------------------------------------------
| mean/                                  |           |
|    agent/rollout/ep_len_mean           | 200       |
|    agent/rollout/ep_rew_mean           | -1.31e+03 |
|    agent/rollout/ep_rew_wrapped_mean   | -31.8     |
|    agent/time/fps                      | 2.4e+03   |
|    agent/time/iterations               | 1         |
|    agent/time/time_elapsed             | 0       

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -1.33e+03    |
|    agent/rollout/ep_rew_wrapped_mean | -32.3        |
|    agent/time/fps                    | 1944         |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 1            |
|    agent/time/total_timesteps        | 4096         |
|    agent/train/approx_kl             | 0.0040343488 |
|    agent/train/clip_fraction         | 0.0299       |
|    agent/train/clip_range            | 0.2          |
|    agent/train/entropy_loss          | -1.41        |
|    agent/train/explained_variance    | 0.214        |
|    agent/train/learning_rate         | 0.0003       |
|    agent/train/loss                  | 0.226        |
|    agent/train/n_updates             | 10           |
|    agent/tra

...

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -1.28e+03    |
|    agent/rollout/ep_rew_wrapped_mean | -38.4        |
|    agent/time/fps                    | 1739         |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 1            |
|    agent/time/total_timesteps        | 10240        |
|    agent/train/approx_kl             | 0.0038231881 |
|    agent/train/clip_fraction         | 0.0264       |
|    agent/train/clip_range            | 0.2          |
|    agent/train/entropy_loss          | -1.41        |
|    agent/train/explained_variance    | 0.675        |
|    agent/train/learning_rate         | 0.0003       |
|    agent/train/loss                  | 1.36         |
|    agent/train/n_updates             | 40           |
|    agent/tra

Training reward model:   0%|          | 0/3 [00:00<?, ?it/s]

Training agent for 1000 timesteps
-------------------------------------------------------
| raw/                                 |              |
|    agent/rollout/ep_len_mean         | 200          |
|    agent/rollout/ep_rew_mean         | -1.27e+03    |
|    agent/rollout/ep_rew_wrapped_mean | -38.5        |
|    agent/time/fps                    | 2067         |
|    agent/time/iterations             | 1            |
|    agent/time/time_elapsed           | 0            |
|    agent/time/total_timesteps        | 12288        |
|    agent/train/approx_kl             | 0.0045314487 |
|    agent/train/clip_fraction         | 0.0304       |
|    agent/train/clip_range            | 0.2          |
|    agent/train/entropy_loss          | -1.39        |
|    agent/train/explained_variance    | 0.837        |
|    agent/train/learning_rate         | 0.0003       |
|    agent/train/loss                  | 0.968        |
|    agent/train/n_updates             | 50           |
|    agent/tra

{'reward_loss': 0.11039837981973376, 'reward_accuracy': 0.9776785714285714}

After we trained the reward network using the preference comparisons algorithm, we can wrap our environment with that learned reward.

In [3]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper


learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

Now we can train an agent, that only sees those learned reward.

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

learner = PPO(
    policy=MlpPolicy,
    env=learned_reward_venv,
    seed=0,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
    n_steps=64,
)
learner.learn(1000)  # Note: set to 100000 to train a proficient expert

Then we can evaluate it using the original reward.

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(learner.policy, venv, 10)
print(reward)

-1057.3085665
